In [3]:
# Re-import necessary libraries after execution state reset
import requests
import pandas as pd
import datetime as dt # library for handling date and time objects

# Define API details
DMI_URL = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'
api_key = '56642815-d535-418b-bda2-30a8a8ef4999'

# Specify the desired start and end time
start_time = pd.Timestamp(2002, 1, 1)
end_time = pd.Timestamp(2024, 12, 31)
datetime_str = start_time.tz_localize('UTC').isoformat() + '/' + end_time.tz_localize('UTC').isoformat()

# Station ID
stationIds = ['06102']

# List of required parameters
parameterIds = [
    "temp_min_past1h", "temp_max_past1h", "temp_mean_past1h", 
    "temp_grass_mean_past1h", "temp_soil_min_past1h", "temp_soil_max_past1h", "temp_soil_mean_past1h",
    "humidity_past1h", "pressure", "wind_dir_past1h", "wind_min_past1h", 
    "wind_gust_always_past1h", "wind_speed_past1h", "precip_past1h", "precip_dur_past1h",
    "snow_depth_man", "radia_glob_past1h", "sun_last1h_glob"
]

# Fetch data for all parameters
dfs = []
for station in stationIds:
    for parameter in parameterIds:
        # Define API query parameters
        params = {
            'api-key': api_key,
            'datetime': datetime_str,
            'stationId': station,
            'parameterId': parameter,
            'limit': '300000',  # Max limit
        }

        # Send GET request
        response = requests.get(DMI_URL, params=params)

        # Check response status
        if response.status_code == 200:
            json_data = response.json()

            # Extract data into DataFrame
            if 'features' in json_data and json_data['features']:
                dfi = pd.json_normalize(json_data['features'])
                dfi['time'] = pd.to_datetime(dfi['properties.observed'])
                dfi = dfi[['time', 'properties.value', 'properties.stationId', 'properties.parameterId']]
                dfi.columns = [c.replace('properties.', '') for c in dfi.columns]
                dfi = dfi[~dfi.duplicated()]  # Remove duplicates
                dfi = dfi.set_index(['parameterId', 'stationId', 'time'])
                dfi = dfi['value'].unstack(['stationId', 'parameterId'])
                dfs.append(dfi)

# Combine all data into a single DataFrame
if dfs:
    df = pd.concat(dfs, axis='columns').sort_index()
    # Resample the DataFrame to hourly by taking the mean
    df = df.resample("H").mean()
    df.to_csv("dmi_weather_data.csv")  # Save to CSV
else:
    print("No data retrieved. Check API response and station/parameter configuration.")


In [4]:
df.head() # Show first rows

stationId                           06102                                   \
parameterId               temp_min_past1h temp_max_past1h temp_mean_past1h   
time                                                                         
2002-01-01 00:00:00+00:00            -4.4            -3.0             -3.5   
2002-01-01 01:00:00+00:00            -8.2            -4.4             -6.6   
2002-01-01 02:00:00+00:00            -8.9            -7.2             -7.9   
2002-01-01 03:00:00+00:00            -9.3            -7.9             -8.4   
2002-01-01 04:00:00+00:00            -9.3            -7.2             -8.6   

stationId                                                              \
parameterId               temp_grass_mean_past1h temp_soil_min_past1h   
time                                                                    
2002-01-01 00:00:00+00:00                   -4.4                  0.0   
2002-01-01 01:00:00+00:00                   -7.7                  0.0   
2002-01-01 02:00:00+00:00                   -9.9                  0.1   
2002-01-01 03:00:00+00:00                  -10.7                  0.0   
2002-01-01 04:00:00+00:00                  -10.3                  0.0   

stationId                                                             \
parameterId               temp_soil_max_past1h temp_soil_mean_past1h   
time                                                                   
2002-01-01 00:00:00+00:00                  0.1                   0.1   
2002-01-01 01:00:00+00:00                  0.1                   0.1   
2002-01-01 02:00:00+00:00                  0.1                   0.1   
2002-01-01 03:00:00+00:00                  0.1                   0.1   
2002-01-01 04:00:00+00:00                  0.1                   0.1   

stationId                                                           \
parameterId               humidity_past1h pressure wind_dir_past1h   
time                                                                 
2002-01-01 00:00:00+00:00            96.0      NaN           279.0   
2002-01-01 01:00:00+00:00            94.0      NaN           281.0   
2002-01-01 02:00:00+00:00            94.0      NaN           280.0   
2002-01-01 03:00:00+00:00            93.0      NaN           276.0   
2002-01-01 04:00:00+00:00            93.0      NaN           276.0   

stationId                                                          \
parameterId               wind_min_past1h wind_gust_always_past1h   
time                                                                
2002-01-01 00:00:00+00:00             NaN                     NaN   
2002-01-01 01:00:00+00:00             NaN                     NaN   
2002-01-01 02:00:00+00:00             NaN                     NaN   
2002-01-01 03:00:00+00:00             NaN                     NaN   
2002-01-01 04:00:00+00:00             NaN                     NaN   

stationId                                                                    \
parameterId               wind_speed_past1h precip_past1h precip_dur_past1h   
time                                                                          
2002-01-01 00:00:00+00:00               1.9           0.0               0.0   
2002-01-01 01:00:00+00:00               2.2           0.0               0.0   
2002-01-01 02:00:00+00:00               2.2           0.0               0.0   
2002-01-01 03:00:00+00:00               1.6           0.0               0.0   
2002-01-01 04:00:00+00:00               2.1           0.0               0.0   

stationId                                                    
parameterId               radia_glob_past1h sun_last1h_glob  
time                                                         
2002-01-01 00:00:00+00:00               0.0             0.0  
2002-01-01 01:00:00+00:00               0.0             0.0  
2002-01-01 02:00:00+00:00               0.0             0.0  
2002-01-01 03:00:00+00:00               0.0             0.0  
2002-01-01 04:00:00+00:00       

In [5]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Data Count:\n", missing_values[missing_values > 0])

Missing Data Count:
 stationId  parameterId            
06102      temp_min_past1h              5996
           temp_max_past1h              5995
           temp_mean_past1h             5999
           temp_grass_mean_past1h       6155
           temp_soil_min_past1h         5536
           temp_soil_max_past1h         5536
           temp_soil_mean_past1h        5536
           humidity_past1h              6199
           pressure                   151515
           wind_dir_past1h              5711
           wind_min_past1h            191282
           wind_gust_always_past1h     20336
           wind_speed_past1h            5711
           precip_past1h                8485
           precip_dur_past1h          103607
           radia_glob_past1h            5624
           sun_last1h_glob              6030
dtype: int64


In [6]:
df.count()

stationId  parameterId            
06102      temp_min_past1h            195605
           temp_max_past1h            195606
           temp_mean_past1h           195602
           temp_grass_mean_past1h     195446
           temp_soil_min_past1h       196065
           temp_soil_max_past1h       196065
           temp_soil_mean_past1h      196065
           humidity_past1h            195402
           pressure                    50086
           wind_dir_past1h            195890
           wind_min_past1h             10319
           wind_gust_always_past1h    181265
           wind_speed_past1h          195890
           precip_past1h              193116
           precip_dur_past1h           97994
           radia_glob_past1h          195977
           sun_last1h_glob            195571
dtype: int64

In [7]:
# Define threshold for missing data (e.g., remove columns with >50% missing)
missing_threshold = 0.50  # 50%

# Calculate missing data percentage
missing_percent = df.isnull().mean()

# Remove columns with too much missing data
columns_to_drop = missing_percent[missing_percent > missing_threshold].index
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Removed columns due to excessive missing data: {list(columns_to_drop)}")

Removed columns due to excessive missing data: [('06102', 'pressure'), ('06102', 'wind_min_past1h'), ('06102', 'precip_dur_past1h')]


In [8]:
df_cleaned.head() # Show first rows

stationId                           06102                                   \
parameterId               temp_min_past1h temp_max_past1h temp_mean_past1h   
time                                                                         
2002-01-01 00:00:00+00:00            -4.4            -3.0             -3.5   
2002-01-01 01:00:00+00:00            -8.2            -4.4             -6.6   
2002-01-01 02:00:00+00:00            -8.9            -7.2             -7.9   
2002-01-01 03:00:00+00:00            -9.3            -7.9             -8.4   
2002-01-01 04:00:00+00:00            -9.3            -7.2             -8.6   

stationId                                                              \
parameterId               temp_grass_mean_past1h temp_soil_min_past1h   
time                                                                    
2002-01-01 00:00:00+00:00                   -4.4                  0.0   
2002-01-01 01:00:00+00:00                   -7.7                  0.0   
2002-01-01 02:00:00+00:00                   -9.9                  0.1   
2002-01-01 03:00:00+00:00                  -10.7                  0.0   
2002-01-01 04:00:00+00:00                  -10.3                  0.0   

stationId                                                             \
parameterId               temp_soil_max_past1h temp_soil_mean_past1h   
time                                                                   
2002-01-01 00:00:00+00:00                  0.1                   0.1   
2002-01-01 01:00:00+00:00                  0.1                   0.1   
2002-01-01 02:00:00+00:00                  0.1                   0.1   
2002-01-01 03:00:00+00:00                  0.1                   0.1   
2002-01-01 04:00:00+00:00                  0.1                   0.1   

stationId                                                  \
parameterId               humidity_past1h wind_dir_past1h   
time                                                        
2002-01-01 00:00:00+00:00            96.0           279.0   
2002-01-01 01:00:00+00:00            94.0           281.0   
2002-01-01 02:00:00+00:00            94.0           280.0   
2002-01-01 03:00:00+00:00            93.0           276.0   
2002-01-01 04:00:00+00:00            93.0           276.0   

stationId                                                            \
parameterId               wind_gust_always_past1h wind_speed_past1h   
time                                                                  
2002-01-01 00:00:00+00:00                     NaN               1.9   
2002-01-01 01:00:00+00:00                     NaN               2.2   
2002-01-01 02:00:00+00:00                     NaN               2.2   
2002-01-01 03:00:00+00:00                     NaN               1.6   
2002-01-01 04:00:00+00:00                     NaN               2.1   

stationId                                                                  
parameterId               precip_past1h radia_glob_past1h sun_last1h_glob  
time                                                                       
2002-01-01 00:00:00+00:00           0.0               0.0             0.0  
2002-01-01 01:00:00+00:00           0.0               0.0             0.0  
2002-01-01 02:00:00+00:00           0.0               0.0             0.0  
2002-01-01 03:00:00+00:00           0.0               0.0             0.0  
2002-01-01 04:00:00+00:00           0.0               0.0             0.0

In [9]:
# Step 1: Identify missing values before processing
missing_before = df_cleaned.isnull().sum()
print(f"Missing values before processing:\n{missing_before[missing_before > 0]}\n")

# Step 2: Apply linear interpolation for gaps up to 12 hours
df_cleaned = df_cleaned.interpolate(method='linear', limit=12, limit_direction='both')

# Step 3: Add helper columns for seasonal mean imputation
df_cleaned["hour"] = df_cleaned.index.hour
df_cleaned["dayofyear"] = df_cleaned.index.dayofyear
df_cleaned["month"] = df_cleaned.index.month
df_cleaned["day"] = df_cleaned.index.day

# Step 4: Remove February 29th
df_cleaned = df_cleaned[~((df_cleaned["month"] == 2) & (df_cleaned["day"] == 29))]

# Step 5: Fill remaining missing values using seasonal means
df_cleaned = df_cleaned.groupby(["dayofyear", "hour"]).transform(lambda x: x.fillna(x.mean()))

# Step 6: Remove helper columns after filling
df_cleaned.drop(columns=["hour", "dayofyear", "month", "day"], errors="ignore", inplace=True)

# Step 7: Save the cleaned dataset to a new CSV file
df_cleaned.to_csv("dmi_weather_data_cleaned.csv")

# Step 7: Identify missing values after processing
missing_after = df_cleaned.isnull().sum()
print(f"Missing values after processing:\n{missing_after[missing_after > 0]}")
print("\nData cleaning complete!")


Missing values before processing:
stationId  parameterId            
06102      temp_min_past1h             5996
           temp_max_past1h             5995
           temp_mean_past1h            5999
           temp_grass_mean_past1h      6155
           temp_soil_min_past1h        5536
           temp_soil_max_past1h        5536
           temp_soil_mean_past1h       5536
           humidity_past1h             6199
           wind_dir_past1h             5711
           wind_gust_always_past1h    20336
           wind_speed_past1h           5711
           precip_past1h               8485
           radia_glob_past1h           5624
           sun_last1h_glob             6030
dtype: int64

Missing values after processing:
Series([], dtype: int64)

Data cleaning complete!
